# Real Data Cleaning
This is the latest file April 24 where I merged the two data sets.

In [485]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('finca_raiz_data.csv')
df2 = pd.read_csv('finca_raiz_data_2.csv')

In [486]:
df = pd.concat([df1, df2], axis=0)
df.shape

(1400, 27)

## Fix Características

In [487]:
df['Características'].fillna('[]').apply(eval)

0                                                     []
1      [En conjunto cerrado, Garaje(s), Parqueadero V...
2                                                     []
3                                                     []
4      [Circuito cerrado de TV, En conjunto cerrado, ...
                             ...                        
592    [Parqueadero Visitantes, Balcón, Cocina Integr...
593    [Balcón, Piso en Baldosa /  Mármol, Colegios /...
594    [Acceso Pavimentado, Vivienda Bifamiliar, Baño...
595    [Acceso Pavimentado, Árboles frutales, Canchas...
596    [Canchas Deportivas, Baño Auxiliar, Shut de ba...
Name: Características, Length: 1400, dtype: object

In [488]:
from sklearn.preprocessing import MultiLabelBinarizer

df['Características'] = df['Características'].fillna('[]').apply(eval)

mlb = MultiLabelBinarizer()
mlb.fit(df['Características'])
carac_df = pd.DataFrame(mlb.transform(df['Características']), columns=mlb.classes_)

to_drop = []
for i in carac_df.columns:
    if 'hace' in i or '$' in i or i[0].isnumeric() or 'inmobili' in i.lower() or 'SAS' in i or 'GRUPO' in i or 'privada' in i.lower() or 'constru' in i.lower():
        to_drop.append(i)
carac_df = carac_df.drop(to_drop, axis=1)

In [489]:
to_drop = []
for col in carac_df.columns:
    if carac_df[col].value_counts(normalize=True).loc[0] > .6:
        to_drop.append(col)
        
carac_df = carac_df.drop(to_drop, axis=1).reset_index()
df = df.reset_index()

In [490]:
df.drop('Características', axis=1, inplace=True)
df = pd.concat([df, carac_df], axis=1)

In [491]:
df = df.drop('index', axis=1)
df = df.drop('Unnamed: 0', axis=1)

In [492]:
to_drop = []

for row in df.isna().sum().index:
    if df.isna().sum().loc[row] > 400:
        to_drop.append(row)

df = df.drop(to_drop, axis=1)

In [493]:
df = df.drop_duplicates(subset=['Enlace'])

In [494]:
df = df.drop('Acción', axis=1)

In [495]:
df['Antigüedad'] = df['Antigüedad'].apply(lambda x: 'M' + str(x) if not str(x)[0].isnumeric() else x)

In [496]:
df = df.drop('Departamento', axis=1)

In [497]:
df = df.drop(['Anuncio publicado por:'], axis=1)

In [498]:
df['Precio m²'] = df['Precio m²'].str.extract('(.*),')

In [499]:
df['Precio m²'] = df['Precio m²'].str.replace('.', '')

C:\Users\HaydenH\AppData\Local\Temp\ipykernel_15716\197371727.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Precio m²'] = df['Precio m²'].str.replace('.', '')


In [500]:
df['Precio (COP)'] = df['Precio (COP)'].str.replace('.', '')

C:\Users\HaydenH\AppData\Local\Temp\ipykernel_15716\237106218.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Precio (COP)'] = df['Precio (COP)'].str.replace('.', '')


In [501]:
df['Administración'] = df['Administración'].str.replace('.', '').str.replace(' COP', '')

C:\Users\HaydenH\AppData\Local\Temp\ipykernel_15716\2464483055.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Administración'] = df['Administración'].str.replace('.', '').str.replace(' COP', '')


In [502]:
pd.options.display.max_columns=None

In [503]:
# df = df.drop('Barrio', axis=1)

In [505]:
df = df.dropna()

In [506]:
df = df[~df['Habitaciones'].str.contains('digo', na=False)]
df = df[~df['Baños'].str.contains('digo', na=False)]
df['Área construída'] = df['Área construída'].str.replace(',', '.')
df['Área privada'] = df['Área privada'].str.replace(',', '.')
df = df[~df['Estrato'].str.contains('digo', na=False)]
df = df[~df['Estrato'].str.contains('Campestr', na=False)]
df['Administración'] = df['Administración'].replace('No definida', 0)
df = df[~df['Administración'].str.contains('digo', na=False)]
df = df[~df['No. Fotos'].str.contains('general', na=False)]


In [507]:
df['Habitaciones'] = df['Habitaciones'].astype('int16')
df['Baños'] = df['Baños'].astype('int8')
df['Área construída'] = df['Área construída'].astype('float32')
df['Área privada'] = df['Área privada'].astype('float32')
df['Estrato'] = df['Estrato'].astype('int8')
df['Administración'] = df['Administración'].astype('int64')
df['Precio m²'] = df['Precio m²'].astype('int64')
df['Precio (COP)'] = df['Precio (COP)'].astype('int64')
df['No. Fotos'] = df['No. Fotos'].astype('int64')

In [510]:
df.to_pickle('colombia_real_estate.pkl')